## Load necessary modules

In [ ]:
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn_pandas import (
    DataFrameMapper, 
    FunctionTransformer
)
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import random
from sklearn.tree import DecisionTreeClassifier

## Load necessary data

In [ ]:
iris = load_iris()
X = iris["data"]
y = iris["target"]
feature_names = [col.replace(" ", "_").replace("_(cm)", "") 
                 for col in (iris["feature_names"] + ["species"])]
class_names = {0.0: "setosa", 1.0: "versicolor", 2.0: "virginica"}

# convert to data frame purely for showing the data
iris_df = pd.DataFrame(np.column_stack((X, y.reshape(-1, 1)))
                       , columns=feature_names)

# convert species from numeric to categorical
iris_df["species"] = iris_df["species"].map(class_names)

iris_df.head()

### For educational purposes only, let's replace some values with `NaN`

In [ ]:
# initialize random number generator
random.seed(2019)

# generate list of random integers 
random_ints = [random.randrange(0, len(iris_df)) for _ in range(20)]

# for these random index values, replace their real values with NaN
iris_df.loc[random_ints, "sepal_length"] = np.nan
iris_df.loc[random_ints, "sepal_width"] = np.nan

# manually force the first "sepal_width" value to also be NaN
iris_df.loc[0, "sepal_width"] = np.nan

iris_df.head(10)

## Create function to identify records that have `NaN` values

This will help us pick up on whether or not having a `NaN` value is helpful information in our model

In [ ]:
def is_missing(x):
    """IDs records that contain missing (NaN) values"""
    if pd.isna(x):
        return 1
    else:
        return 0

## Do not create an object that imputes median values

Unfortunately, `sklearn_pandas` is still under development. The usage of the following:

```python
impute_median = SimpleImputer(missing_values=np.nan, strategy="median")
```

inside `DataFrameMapper` will result in all columns that use the `impute_median` transformer to have the same value.

Instead, do a gut check by manually printing out the median values.

In [ ]:
print(f"Median value for sepal length: {iris_df['sepal_length'].median()}")
print(f"Median value for sepal width: {iris_df['sepal_width'].median()}")

## Store preprocessing steps in one object

![kid transformer](visuals/transformer.gif)

We'll store these steps in a `DataFrameMapper` object. A `DataFrameMapper` maps `pandas` data frame column subsets to their own transformations. This is abstraction is useful since we'll always be applying the same preprocessing steps to both our training and testing sets.

The key here is we need to specify a specific column, pass it's "transformer" (i.e. `SimpleImputer`, `OneHotEncoder`, `StandardScaler`), and determinie if the transformation belongs in it's own new column or if it's more appropriate for the transformed column to overwrite the input column.

Here, we'll create `mapper` object to do the following:

* flag which records contain `NaN` values in the `sepal_length` feature;
* impute the median value for `NaN` values in the `sepal_length` feature;
* flag which records contain `NaN` values in the `sepal_width` feature;
* impute the median value for `NaN` values in the `sepal_width` feature; and
* return all other columns - `petal_length` & `petal_width` - as is.


In [ ]:
mapper = DataFrameMapper(features=[
    (["sepal_length"], FunctionTransformer(is_missing), {"alias": "sepal_length_missing"}),
    (["sepal_length"], SimpleImputer(missing_values=np.nan, strategy="median")),
    (["sepal_width"], FunctionTransformer(is_missing), {"alias": "sepal_width_missing"}),
    (["sepal_width"], SimpleImputer(missing_values=np.nan, strategy="median")),
],
                         default=None,
                         df_out=True)

In [ ]:
# just like a model object, 
# we'll fit and transform the DataFrameMapper object onto our data
mapper.fit(iris_df).transform(iris_df).head(10)

Notice how the 9th record in both columns contains a value of `1`. This algins with the output of `iris_df.head(10)`, with both values in `sepal_length` and `sepal_width` containing `NaN` values.

The use of our custom function was honored and bug-free, as only the first record in `sepal_width_missing` has a value of `1` and `sepal_length_missing` has a value of `0`.

All records in the `petal_length`, `petal_width` and `species` columns are returned with no transformation.

## Now let's take this up a level by performing `train_test_split` before any preprocessing and build a `DecisionTreeClassifier` model to classify which species a flower is

In [ ]:
# split data 
X_train, X_test, y_train, y_test = train_test_split(iris_df.drop("species", axis=1),
                                                    iris_df["species"],
                                                    test_size=0.3,
                                                    random_state=2019)

In [ ]:
# build classifier
dt_clf = DecisionTreeClassifier(random_state=2019,
                                min_samples_leaf=30,
                                criterion="gini",
                                min_samples_split=2)

In [ ]:
# build pipeline
pipe = Pipeline(steps=[
    ("dataprep", mapper),
    ("model", dt_clf)
])

Fit `X_train` and `y_train` onto the `pipe` object

In [ ]:
pipe.fit(X_train, y_train)

Use the `pipe` object to make predictions on `X_test`

In [ ]:
y_pred = pipe.predict(X_test)

Examine the first 10 predictions of our multi-class classifier!

In [ ]:
y_pred[0:10]

Compare with `y_test`

In [ ]:
y_test[0:10]

## TODO

* Make multi class ROC Curve: https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html